## Parte 1: Validación de Datos
1.1 Análisis Exploratorio Inicial


In [0]:
%sql

SHOW TABLES IN workspace.operations;


##a) Conteo de registros por tabla

- ¿Cuántas transacciones hay en total? 

In [0]:
%sql
-- 1. Transacciones totales
SELECT COUNT(*) AS total_transacciones
FROM workspace.operations.gold_fact_sales_product;

- ¿Cuántos productos activos existen? 

In [0]:
%sql
-- 2. Productos activos
SELECT COUNT(*) AS productos_activos
FROM workspace.operations.gold_dim_product
WHERE STATUS = 'A';


- ¿Cuántas tiendas activas hay? 

In [0]:
%sql
-- 3. Tiendas activas
SELECT COUNT(*) AS tiendas_activas
FROM workspace.operations.gold_dim_organization
WHERE STORE_ACTIVE = 'Y';

- ¿Cuántos proveedores activos?


In [0]:
%sql
-- 4. Proveedores activos
SELECT COUNT(*) AS proveedores_activos
FROM workspace.operations.gold_dim_supplier
WHERE SUP_STATUS = 'A';


##b) Validación de rango de fechas

In [0]:
%sql
-- Rango de fechas en la tabla de ventas
SELECT 
    MIN(BUSINESS_DATE) AS fecha_minima,
    MAX(BUSINESS_DATE) AS fecha_maxima
FROM workspace.operations.gold_fact_sales_product;


## c) Validación de integridad referencial

In [0]:
%sql
-- Verificar integridad SK_PRODUCT
SELECT COUNT(*) AS registros_sin_producto
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_product p
ON f.SK_PRODUCT = p.SK_PRODUCT
WHERE p.SK_PRODUCT IS NULL;


In [0]:
%sql
-- Verificar integridad SK_ORGANIZATION
SELECT COUNT(*) AS registros_sin_tienda
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_organization o
ON f.SK_ORGANIZATION = o.SK_ORGANIZATION
WHERE o.SK_ORGANIZATION IS NULL;

In [0]:
%sql
-- Verificar integridad SK_SUPPLIER
SELECT COUNT(*) AS registros_sin_proveedor
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_supplier s
ON f.SK_SUPPLIER = s.SK_SUPPLIER
WHERE s.SK_SUPPLIER IS NULL;

In [0]:
%sql
-- Verificar integridad SK_CHANNEL
SELECT COUNT(*) AS registros_sin_canal
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_channel c
ON f.SK_CHANNEL = c.SK_CHANNEL
WHERE c.SK_CHANNEL IS NULL;


In [0]:
%sql
-- Verificar integridad SK_CURRENCY
SELECT COUNT(*) AS registros_sin_moneda
FROM workspace.operations.gold_fact_sales_product f
LEFT JOIN workspace.operations.gold_dim_currency cur
ON f.SK_CURRENCY = cur.SK_CURRENCY
WHERE cur.SK_CURRENCY IS NULL;

## d) Validación de valores nulos y negativos


- Ventas nulas
- Ventas negativas
- Unidades nulas o negativas

In [0]:
%sql
-- Ventas nulas
SELECT COUNT(*) AS ventas_nulas
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE IS NULL;

In [0]:
%sql
-- Ventas negativas
SELECT COUNT(*) AS ventas_negativas
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE < 0;

In [0]:
%sql
-- Unidades inválidas
SELECT COUNT(*) AS unidades_invalidas
FROM workspace.operations.gold_fact_sales_product
WHERE UNITS IS NULL OR UNITS <= 0;


## e) Validación de consistencia de cálculos



In [0]:
%sql
-- Validar TOTAL_SALE + TAX = TOTAL_SALE_TAX
SELECT COUNT(*) AS inconsistencias_total
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE + TAX <> TOTAL_SALE_TAX;


##Validaciones adicionales

Esto detecta si hay ventas con costo mayor al total (margen negativo), lo cual podría ser error o pérdida.

In [0]:
%sql
SELECT COUNT(*) 
FROM workspace.operations.gold_fact_sales_product
WHERE TOTAL_SALE - TOTAL_COST < 0;


Esta es una validacion para saber si el TAX (16%) es realmente el 16% del total de la venta

In [0]:
%sql
SELECT COUNT(*) 
FROM workspace.operations.gold_fact_sales_product
WHERE ROUND(TAX,2) <> ROUND(TOTAL_SALE*0.16,2);



##1.2 Documento de Hallazgos


## Resumen de Validación de Datos

✅ Validaciones que pasaron exitosamente
⚠️ Problemas encontrados 
📋 Recomendaciones para mejorar la calidad de datos

a) Conteo de registros por tabla ✅ 

- **Transacciones totales:** 432,987  
- **Productos activos:** 749  
- **Tiendas activas:** 94  
- **Proveedores activos:** 7  

b) Validación de rango de fechas ✅ 

- Todas las transacciones están dentro del rango **julio - septiembre 2025**.  

c) Validación de integridad referencial ✅

- Todas las claves foráneas (`SK_PRODUCT`, `SK_ORGANIZATION`, `SK_SUPPLIER`, `SK_CHANNEL`, `SK_CURRENCY`) tienen correspondencia en sus tablas de dimensiones.  
- **Resultado:** 0 registros sin correspondencia  
Un resultado de 0 indica que **todas las claves foráneas son válidas**, es decir, no hay registros huérfanos en la tabla de ventas. 

d) Validación de valores nulos y negativos ✅

- **Ventas nulas:** 0  -> No se encontraron ventas nulas
- **Ventas negativas:** 0  -> No se encontraron ventas negativas
- **Unidades inválidas (nulas o <=0):** 0  -> No se encontraron unidades invalidas o nulas

e) Validación de consistencia de cálculos ⚠️

- **TOTAL_SALE + TAX = TOTAL_SALE_TAX**  
  - Se encontraron **95,687 registros** que no cumplen esta relación.  

- **TAX = 16% de TOTAL_SALE**  
  - Ningún registro coincide exactamente con la regla del 16%.   
- Una parte de los registros no cumple la relación TOTAL_SALE + TAX = TOTAL_SALE_TAX.  
- El impuesto (TAX) no sigue exactamente el 16% sobre la venta, probablemente porque los datos son generados como ejemplo y no reflejan cálculos reales.  

> 📋 Recomendación:  
- Se sugiere que **los datos crudos** se almacenen tal como se reciben, y que **cálculos como el IVA y el total con impuestos se realicen automáticamente en los procesos ETL**.  
- Esto garantiza consistencia y evita errores de cálculo manual.

---

## ✅ Conclusión General

Los datos del trimestre presentan buena calidad en cuanto a:  
- Conteo de registros  
- Fechas  
- Integridad referencial  
- Valores nulos y negativos  

El principal punto a mejorar son los cálculos derivados (impuesto y total con impuestos), los cuales deben automatizarse para garantizar métricas consistentes en el análisis de negocio.
